In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from Utils.DataLoader import DataLoader
from Utils.DataTransformer import DataTransformer
from Utils.DuckDb.DuckDb import DuckDb
from Utils.DatasetProcessor.DatasetProcessorUtils import DatasetProcessorUtils
from datetime import datetime
import pandas as pd
from typing import List
from Utils import PandasNotebookConfigs
import project_config
import numpy as np
import matplotlib.pyplot as plt
import joblib
from datetime import datetime
from Utils.Datetime import DatetimeUtils


In [11]:
PandasNotebookConfigs.config()

Ajustando display.float.format para %.4f


In [12]:
DUCK_DB = DuckDb()
DATA_TRANSFORMER = DataTransformer()
DATA_PROCESSOR_UTILS = DatasetProcessorUtils(DUCK_DB)
DATETIME_UTILS = DatetimeUtils()

In [13]:
PRE_PREPARED_DATA_TABLE = project_config.PRE_PREPARED_DATA_TABLE
PREDICTED_DATA_TABLE = f'predicted_dataset_{DATETIME_UTILS.get_datetime_string_identifier()}'

In [14]:
PREDICTED_DATA_TABLE

'predicted_dataset_2025_03_26_20h52m'

In [15]:
df = DUCK_DB.load_table_to_predict(
    PRE_PREPARED_DATA_TABLE,
    start_from_safra=project_config.MAX_SAFRA_TO_CONSIDER_ON_DATA_PREPARE
)

In [16]:
# input('Prometo tirar isso aqui!')
# df = df[:10].copy()

In [17]:
df = DATA_PROCESSOR_UTILS.calc_remaining_days(df)

In [18]:
df.head()

,cost,msno,safra,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,...,num_25-1M,num_50-2M,num_50-1M,num_75-2M,num_75-1M,num_985-2M,num_985-1M,num_100-2M,num_100-1M,remaining_days
0,67.3556,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,201611,128,71,50,101,470,488,148668,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0000
1,80.6741,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,201612,193,88,98,142,855,979,256812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0000
2,69.3364,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,201701,135,69,78,147,438,696,157868,...,193.0000,71.0000,88.0000,50.0000,98.0000,101.0000,142.0000,470.0000,855.0000,31.0000
3,68.9275,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,201702,116,61,58,114,468,642,156533,...,135.0000,88.0000,69.0000,98.0000,78.0000,142.0000,147.0000,855.0000,438.0000,28.0000
4,66.1132,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,201611,59,23,11,21,520,545,133337,...,32.0000,28.0000,12.0000,18.0000,14.0000,19.0000,20.0000,665.0000,755.0000,50.0000


In [19]:
# Data to predict
df['safra'].value_counts()

safra
201611    227022
201612    211127
201701    205849
201702    173975
Name: count, dtype: int64

In [20]:
len(df)

817973

In [21]:
# Treating DF
df.isna().sum().T

cost                                       0
msno                                       0
safra                                      0
num_25                                     0
num_50                                     0
num_75                                     0
num_985                                    0
num_100                                    0
num_unq                                    0
total_secs                                 0
total_hours                                0
msno_1                                     0
payment_method_id                          0
payment_plan_days                          0
plan_list_price                            0
actual_amount_paid                         0
is_auto_renew                              0
is_cancel                                  0
safra_1                                    0
transaction_date_year                      0
transaction_date_month                     0
transaction_date_day                       0
transactio

In [22]:
count = 0

def fill_out_historical_data(df: pd.DataFrame) -> pd.DataFrame:
    past_safras_to_consider = project_config.SAFRAS_TO_CONSIDER_ON_HISTORICAL_DATA

    # Key: past col
    # Value: "actual" col
    historical_cols = {}

    for col in project_config.COLUMNS_TO_GET_HISTORICAL_DATA:
        for s in past_safras_to_consider:
            historical_cols[f'{col}{s}M'] = col

    df_size = len(df)

    def __process(row: pd.Series):
        global count
        if count % 1500 == 0:
            print(f'-> {count} / {df_size}')

        count += 1

        edited = False
        for past_col, col in historical_cols.items():
            if pd.isna(row[past_col]):
                row[past_col] = int(row[col])
                edited = True

        row['_filled_out_past_info'] = edited

        return row

    df = df.apply(lambda row: __process(row), axis=1)

    return df

df = fill_out_historical_data(df)

-> 0 / 817973
-> 1500 / 817973
-> 3000 / 817973
-> 4500 / 817973
-> 6000 / 817973
-> 7500 / 817973
-> 9000 / 817973
-> 10500 / 817973
-> 12000 / 817973
-> 13500 / 817973
-> 15000 / 817973
-> 16500 / 817973
-> 18000 / 817973
-> 19500 / 817973
-> 21000 / 817973
-> 22500 / 817973
-> 24000 / 817973
-> 25500 / 817973
-> 27000 / 817973
-> 28500 / 817973
-> 30000 / 817973
-> 31500 / 817973
-> 33000 / 817973
-> 34500 / 817973
-> 36000 / 817973
-> 37500 / 817973
-> 39000 / 817973
-> 40500 / 817973
-> 42000 / 817973
-> 43500 / 817973
-> 45000 / 817973
-> 46500 / 817973
-> 48000 / 817973
-> 49500 / 817973
-> 51000 / 817973
-> 52500 / 817973
-> 54000 / 817973
-> 55500 / 817973
-> 57000 / 817973
-> 58500 / 817973
-> 60000 / 817973
-> 61500 / 817973
-> 63000 / 817973
-> 64500 / 817973
-> 66000 / 817973
-> 67500 / 817973
-> 69000 / 817973
-> 70500 / 817973
-> 72000 / 817973
-> 73500 / 817973
-> 75000 / 817973
-> 76500 / 817973
-> 78000 / 817973
-> 79500 / 817973
-> 81000 / 817973
-> 82500 / 817973
->

In [23]:
def print_cols_and_historical_cols(df: pd.DataFrame):
    cols = ['_filled_out_past_info']
    for col in project_config.COLUMNS_TO_GET_HISTORICAL_DATA:
        cols.append(col)

        for sfr in project_config.SAFRAS_TO_CONSIDER_ON_HISTORICAL_DATA:
            cols.append(f'{col}{sfr}M')

    return df[cols]

print_cols_and_historical_cols(df).head().T

,0,1,2,3,4
_filled_out_past_info,True,True,False,False,False
num_unq,488,979,696,642,545
num_unq-2M,488.0000,979.0000,488.0000,979.0000,707.0000
num_unq-1M,488.0000,979.0000,979.0000,696.0000,791.0000
total_secs,148668,256812,157868,156533,133337
total_secs-2M,148668.0000,256812.0000,148668.0000,256812.0000,173410.0000
total_secs-1M,148668.0000,256812.0000,256812.0000,157868.0000,190726.0000
num_25,128,193,135,116,59
num_25-2M,128.0000,193.0000,128.0000,193.0000,53.0000
num_25-1M,128.0000,193.0000,193.0000,135.0000,32.0000


In [24]:
# Treating DF
df.isna().sum().T

cost                                       0
msno                                       0
safra                                      0
num_25                                     0
num_50                                     0
num_75                                     0
num_985                                    0
num_100                                    0
num_unq                                    0
total_secs                                 0
total_hours                                0
msno_1                                     0
payment_method_id                          0
payment_plan_days                          0
plan_list_price                            0
actual_amount_paid                         0
is_auto_renew                              0
is_cancel                                  0
safra_1                                    0
transaction_date_year                      0
transaction_date_month                     0
transaction_date_day                       0
transactio

In [25]:
df = DATA_PROCESSOR_UTILS.process_categories(df)

Adicionando coluna payment_method_id_0
Adicionando coluna payment_method_id_1
Adicionando coluna payment_method_id_2
Adicionando coluna payment_method_id_3
Adicionando coluna payment_method_id_4
Adicionando coluna payment_method_id_5
Adicionando coluna payment_method_id_6
Adicionando coluna payment_method_id_7
Adicionando coluna payment_method_id_8
Adicionando coluna payment_method_id_9
Adicionando coluna payment_method_id_13
Adicionando coluna payment_method_id_22
Adicionando coluna payment_method_id_24
Adicionando coluna payment_method_id_25
Adicionando coluna city_0
Adicionando coluna city_2
Adicionando coluna registered_via_0
Adicionando coluna registered_via_1
Adicionando coluna registered_via_2
Adicionando coluna registered_via_5
Adicionando coluna registered_via_6
Adicionando coluna registered_via_8
Adicionando coluna registered_via_10
Adicionando coluna registered_via_11
Adicionando coluna registered_via_12
Adicionando coluna registered_via_14
Adicionando coluna registered_via_

In [26]:
df = DATA_PROCESSOR_UTILS.split_safras(df)

In [27]:
df_to_predict = df[project_config.COLUMNS_USED_ON_TRAIN_AND_PREDICTION]

In [28]:
df_to_predict.head()

,registration_init_time_day,registration_init_time_day_of_year,registration_init_time_year,transaction_date_day,transaction_date_day_of_week,transaction_date_day_of_year,transaction_date_month,membership_expire_date_day,membership_expire_date_day_of_year,membership_expire_date_month,...,payment_method_id_38,payment_method_id_39,payment_method_id_4,payment_method_id_40,payment_method_id_41,payment_method_id_5,payment_method_id_6,payment_method_id_7,payment_method_id_8,payment_method_id_9
0,8,159,2014,16,2,321,11,15,350,12,...,False,False,False,False,True,False,False,False,False,False
1,8,159,2014,15,3,350,12,15,15,1,...,False,False,False,False,True,False,False,False,False,False
2,8,159,2014,15,6,15,1,15,46,2,...,False,False,False,False,True,False,False,False,False,False
3,8,159,2014,15,2,46,2,15,74,3,...,False,False,False,False,True,False,False,False,False,False
4,18,322,2010,30,2,335,11,19,19,1,...,False,True,False,False,False,False,False,False,False,False


In [29]:
from sklearn.ensemble import RandomForestClassifier

model: RandomForestClassifier = joblib.load(project_config.SELECTED_MODEL)
print(f'Carregado modelo {project_config.SELECTED_MODEL}: {model}')

Carregado modelo ./models/random_forest_model_2025-03-24-21h-56m.joblib: RandomForestClassifier(bootstrap=False, max_depth=30, max_features='log2',
                       min_samples_split=10, random_state=42)


In [30]:
predicted_is_churn = model.predict(df_to_predict)
predicted_is_churn_proba = model.predict_proba(df_to_predict)

In [31]:
predicted_is_churn[:5], predicted_is_churn_proba[:5, 0]

(array([False,  True,  True,  True,  True]),
 array([0.68328901, 0.38254762, 0.39217147, 0.44840322, 0.        ]))

In [32]:
df['predicted_is_churn'] = predicted_is_churn

df['predicted_is_churn_proba_false'] = predicted_is_churn_proba[:, 0]
df['predicted_is_churn_proba_true'] = predicted_is_churn_proba[:, 1]

In [33]:
df.head()

,cost,msno,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,total_hours,...,registered_via_15,registered_via_16,registered_via_17,registered_via_18,registered_via_19,safra_year,safra_month,predicted_is_churn,predicted_is_churn_proba_false,predicted_is_churn_proba_true
0,67.3556,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,128,71,50,101,470,488,148668,41.2967,...,False,False,False,False,False,2016.0000,11.0000,False,0.6833,0.3167
1,80.6741,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,193,88,98,142,855,979,256812,71.3367,...,False,False,False,False,False,2016.0000,12.0000,True,0.3825,0.6175
2,69.3364,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,135,69,78,147,438,696,157868,43.8522,...,False,False,False,False,False,2017.0000,1.0000,True,0.3922,0.6078
3,68.9275,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,116,61,58,114,468,642,156533,43.4814,...,False,False,False,False,False,2017.0000,2.0000,True,0.4484,0.5516
4,66.1132,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,59,23,11,21,520,545,133337,37.0381,...,False,False,False,False,False,2016.0000,11.0000,True,0.0000,1.0000


In [34]:
DUCK_DB.create_database_table(
    df, PREDICTED_DATA_TABLE
)

In [35]:
DUCK_DB.truncate_table(
    PREDICTED_DATA_TABLE
)

In [36]:
DUCK_DB.upload_dataframe_to_duck_db(
    df, PREDICTED_DATA_TABLE
)

2025-03-26 20:57:27.581372 Inserindo 817973 registros na tabela predicted_dataset_2025_03_26_20h52m


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2025-03-26 20:57:30.870320 Inseridos com sucesso


In [37]:
# DUCK_DB_UTILS.get_connection().close()

In [38]:
DUCK_DB.commit()